In [8]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy import stats
from tqdm import tqdm

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats

In [9]:
def calc_change_rate(x):
    change = (np.diff(x) / x[:-1]).values
    change = change[np.nonzero(change)[0]]
    change = change[~np.isnan(change)]
    change = change[change != -np.inf]
    change = change[change != np.inf]
    return np.mean(change)

In [10]:
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

In [11]:
submission = pd.read_csv('data/sample_submission.csv', index_col='seg_id')
submission

,time_to_failure
seg_id,
seg_00030f,0
seg_0012b5,0
seg_00184e,0
seg_003339,0
seg_0042cc,0
...,...
seg_ff4236,0
seg_ff7478,0
seg_ff79d9,0


In [12]:
X= pd.DataFrame(dtype=np.float64, index=submission.index)
X

""
seg_id
seg_00030f
seg_0012b5
seg_00184e
seg_003339
seg_0042cc
...
seg_ff4236
seg_ff7478
seg_ff79d9


In [13]:
segment = 0
c = 0
for i, segment in enumerate(tqdm(X.index)):
#       if c<=2:
        c += 1
        url = 'data/test/{}.csv'.format(segment)
        seg = pd.read_csv(url)
        
        x = pd.Series(seg['acoustic_data'].values)  
        #Một số tổng hợp thông thường: mean, std, min, max
        X.loc[segment, 'mean'] = x.mean()
        X.loc[segment, 'std'] = x.std()
        X.loc[segment, 'abs_mean'] = np.abs(x).mean()
        X.loc[segment, 'abs_std'] = np.abs(x).std()
        X.loc[segment, 'max'] = x.max()
        X.loc[segment, 'min'] = x.min()
        X.loc[segment, 'abs_max'] = np.abs(x).max()
        X.loc[segment, 'abs_min'] = np.abs(x).min()

        #Chênh lệch trung bình giữa các giá trị liên tiếp về giá trị tuyệt đối và giá trị phần trăm;
        X.loc[segment, 'mean_change_abs'] = np.mean(np.diff(x))
        X.loc[segment, 'mean_change_rate'] = calc_change_rate(x)

        #Lấy mẫu n phần từ đầu tiến và n phần tử cuối cùng rồi tính toán 
        X.loc[segment, 'std_first_50000'] = x[:50000].std()
        X.loc[segment, 'std_last_50000'] = x[-50000:].std()
        X.loc[segment, 'std_first_10000'] = x[:10000].std()
        X.loc[segment, 'std_last_10000'] = x[-10000:].std()

        X.loc[segment, 'avg_first_50000'] = x[:50000].mean()
        X.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
        X.loc[segment, 'avg_first_10000'] = x[:10000].mean()
        X.loc[segment, 'avg_last_10000'] = x[-10000:].mean()

        X.loc[segment, 'min_first_50000'] = x[:50000].min()
        X.loc[segment, 'min_last_50000'] = x[-50000:].min()
        X.loc[segment, 'min_first_10000'] = x[:10000].min()
        X.loc[segment, 'min_last_10000'] = x[-10000:].min()

        X.loc[segment, 'max_first_50000'] = x[:50000].max()
        X.loc[segment, 'max_last_50000'] = x[-50000:].max()
        X.loc[segment, 'max_first_10000'] = x[:10000].max()
        X.loc[segment, 'max_last_10000'] = x[-10000:].max()

        X.loc[segment, 'abs_min_first_50000'] = np.abs(x[:50000]).min()
        X.loc[segment, 'abs_min_last_50000'] = np.abs(x[:50000]).min()
        X.loc[segment, 'abs_min_first_10000'] = np.abs(x[:10000]).min()
        X.loc[segment, 'abs_min_last_10000'] = np.abs(x[:10000]).min()

        X.loc[segment, 'abs_max_first_50000'] = np.abs(x[:50000]).max()
        X.loc[segment, 'abs_max_last_50000'] = np.abs(x[:50000]).max()
        X.loc[segment, 'abs_max_first_10000'] = np.abs(x[:10000]).max()
        X.loc[segment, 'abs_max_last_10000'] = np.abs(x[:10000]).max()

        X.loc[segment, 'mean_change_rate_first_50000'] = calc_change_rate(x[:50000])
        X.loc[segment, 'mean_change_rate_last_50000'] = calc_change_rate(x[-50000:])
        X.loc[segment, 'mean_change_rate_first_10000'] = calc_change_rate(x[:10000])
        X.loc[segment, 'mean_change_rate_last_10000'] = calc_change_rate(x[-10000:])

        #Thương, hiệu giữa đồ lớn max và abs(min), đếm các khoảng cách max và min > một ngưỡng 
        X.loc[segment, 'max_to_min'] = x.max() / np.abs(x.min())
        X.loc[segment, 'max_to_min_diff'] = x.max() - np.abs(x.min())
        X.loc[segment, 'count_big'] = len(x[np.abs(x) > 500])
        X.loc[segment, 'sum'] = x.sum()

        #Tính toán Quantile
        X.loc[segment, 'q95'] = np.quantile(x, 0.95)
        X.loc[segment, 'q99'] = np.quantile(x, 0.99)
        X.loc[segment, 'q05'] = np.quantile(x, 0.05)
        X.loc[segment, 'q01'] = np.quantile(x, 0.01)
        X.loc[segment, 'q999'] = np.quantile(x,0.999)
        X.loc[segment, 'q001'] = np.quantile(x,0.001)
        X.loc[segment, 'med'] = x.median()
        X.loc[segment, 'abs_q95'] = np.quantile(np.abs(x), 0.95)
        X.loc[segment, 'abs_q99'] = np.quantile(np.abs(x), 0.99)
        X.loc[segment, 'abs_q05'] = np.quantile(np.abs(x), 0.05)
        X.loc[segment, 'abs_q01'] = np.quantile(np.abs(x), 0.01)

        #trend
        X.loc[segment, 'trend'] = add_trend_feature(x)
        X.loc[segment, 'abs_trend'] = add_trend_feature(x, abs_values=True)


        #Một số chỉ số khác 
        X.loc[segment, 'mad'] = x.mad()
        X.loc[segment, 'kurt'] = x.kurtosis()
        X.loc[segment, 'skew'] = x.skew()
        X.loc[segment, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
        X.loc[segment, 'Hann_window_mean'] = (convolve(x, hann(150), mode='same') / sum(hann(150))).mean()

        #rolling 
        for windows in [10, 100, 1000]:
            x_roll_std = x.rolling(windows).std().dropna().values
            x_roll_mean = x.rolling(windows).mean().dropna().values

            X.loc[segment, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
            X.loc[segment, 'std_roll_std_' + str(windows)] = x_roll_std.std()
            X.loc[segment, 'max_roll_std_' + str(windows)] = x_roll_std.max()
            X.loc[segment, 'min_roll_std_' + str(windows)] = x_roll_std.min()


            X.loc[segment, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
            X.loc[segment, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
            X.loc[segment, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
            X.loc[segment, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
            X.loc[segment, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
            X.loc[segment, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
            X.loc[segment, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
            X.loc[segment, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)

            X.loc[segment, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.diff(x_roll_std))
            X.loc[segment, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.diff(x_roll_mean))

            X.loc[segment, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
            X.loc[segment, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

       

100%|██████████| 2624/2624 [09:04<00:00,  4.82it/s]


In [14]:
X.head()

,mean,std,abs_mean,abs_std,max,min,abs_max,abs_min,mean_change_abs,mean_change_rate,...,q95_roll_std_1000,q99_roll_std_1000,q01_roll_mean_1000,q05_roll_mean_1000,q95_roll_mean_1000,q99_roll_mean_1000,av_change_abs_roll_std_1000,av_change_abs_roll_mean_1000,abs_max_roll_std_1000,abs_max_roll_mean_1000
seg_id,,,,,,,,,,,,,,,,,,,,,
seg_00030f,4.491780,4.893690,5.224607,4.102161,115.0,-75.0,115.0,0.0,0.000027,0.005003,...,8.508256,13.155952,3.889,4.099,4.867,5.000,-3.407406e-05,1.778523e-06,24.717247,5.495
seg_0012b5,4.171153,5.922839,5.198340,5.045369,152.0,-140.0,152.0,0.0,-0.000013,-0.016036,...,10.933346,24.331347,3.644,3.790,4.541,4.739,-6.552189e-06,-1.946309e-07,29.680311,5.009
seg_00184e,4.610260,6.946990,5.597193,6.179525,248.0,-193.0,248.0,0.0,-0.000020,0.037691,...,10.084999,31.075393,4.013,4.215,4.966,5.082,6.882434e-07,8.053691e-08,53.069292,6.234
seg_003339,4.531473,4.114147,4.961487,3.583863,85.0,-93.0,93.0,0.0,0.000047,0.064439,...,6.967421,11.796560,4.032,4.184,4.911,5.051,-3.118098e-07,3.899329e-06,22.803241,5.446
seg_0042cc,4.128340,5.797164,5.070900,4.993617,177.0,-147.0,177.0,0.0,-0.000007,-0.010527,...,10.143349,23.082517,3.534,3.662,4.570,4.870,-1.278093e-06,-2.939597e-06,34.000818,5.027


In [15]:
title.head()

NameError: name 'title' is not defined

In [ ]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler() 
scaler.fit(X) 
X = scaler.transform(X)

In [ ]:
pd.DataFrame(X)

In [ ]:
import pickle

In [ ]:
filename = "SVRModel.pkl"
with open(filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [ ]:
y_predict = pickle_model.predict(X) 
y_predict

In [ ]:
pd.DataFrame(y_predict)

In [ ]:
submission['time_to_failure'] = y_predict

In [ ]:
submission

In [ ]:
submission.to_csv('submission_2')